In [1]:
# coding=utf-8
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import tensorflow as tf
from logging_future import logger

In [3]:
location = "C:/Users/workspace/brad_public_workspace_on_win/non_code_files_brad_public_workspace_on_win/brad_public_workspace_on_win_non_code_files/SH_tongliang/data/11_22_crypto_data/" 

file1 = "res_11_21_adau18z18乘10000000.csv"
file2 = "res_11_21_bchu18z18乘1000.csv"
file3 = "res_11_21_xbtusd乘1.csv"
file4 = "res_11_21_eosu18z18乘100000.csv"
file5 = "res_11_21_ethu18z18乘10000.csv"
file6 = "res_11_21_ltcu18z18乘10000.csv"
file7 = "res_11_21_trxu18z18乘100000000.csv"
file8 = "res_11_21_xrpu18z18乘10000000.csv"

df_ada = pd.read_csv(location + file1, engine="python")
df_bch = pd.read_csv(location + file2, engine="python")
df_btc = pd.read_csv(location + file3, engine="python")
df_eos = pd.read_csv(location + file4, engine="python")
df_eth = pd.read_csv(location + file5, engine="python")
df_ltc = pd.read_csv(location + file6, engine="python")
df_trx = pd.read_csv(location + file7, engine="python")
df_xrp = pd.read_csv(location + file8, engine="python")

In [4]:
# create vars and paras lists:

"""
Here we will use for para in paras_to_optimize:
    ...
"""

# df and df_asst_og are the same
def lstm_all_in_one_optimization(df, df_asst_og, asst, test_train_split, time_step, rnn_unit, batch_size, train_times, lr):
    # Use this code to prevent the kernel from requesting to start again everytime we run the model. 
    tf.reset_default_graph()

    # create func to record paras each time we finish running
    optimization_record = {'test_train_split': [],
                           'time_step': [],
                           'rnn_unit': [],
                           'batch_size': [],
                           'train_times':[],
                           'lr':[],
                           'total_rtrn_pct':[]}
    
    data_len = len(df)
    total_days = data_len
    test_end = total_days
    train_begin = 0
    
    pre_data = df.iloc[:, 4].values  # 取收盘价计算标签
    label = []
    for i in range(1, len(pre_data)):
        label.append(round(pre_data[i] - pre_data[i - 1], 4))

    df.loc[1:, 'label'] = label
    df['label'] = df['label'].shift(-1)
    df.fillna(method='ffill', inplace = True)
    data = df.iloc[:, [1, 2, 3, 5]].values  # 获取特征量及标签，类型为np.ndarray
    # logger.info(data[0:5])

    
    # 参数类别一：
    """
    保持以下参数不变：

    利用过去一天: 2018-11-16 0:00, 来预测下一天：
    time_step = 20
    rnn_unit = 10  # 隐藏层单元数量 ##################调整这个
    input_size = 3  # 输入个数  
    output_size = 1  # 输出个数
    batch_size = 80  # 批量大小  #################调整这个
    train_times = 20  # 训练次数
    lr = 0.001  # 学习率


    test_train_split 参数调优:

    test_train_split   accuracy       信号强度
           0.2           0.55         -55.86
           0.3           0.52         -39.31
           0.4           0.4928        -9.88
           0.5           0.5111       -28.13
           0.6           0.5          -13.87
           0.7           0.4961       -27.13
           0.8           0.4967       -28.58
           0.9           0.5          -9.06

    """

    test_train_split = test_train_split
    time_step = time_step # 20 as default
    split_batch = int(total_days/time_step) 
    time_step_multiple = int(test_train_split*split_batch)
    test_begin = int(test_end - (time_step)*time_step_multiple)
    train_end = test_begin

    # 这里 test_end - test_begin的差一定要是time_step的倍数
#     print ("split_batch: ", split_batch)
#     print ("train_test_split_rate: ", test_train_split)
#     print ("train_begin: ", train_begin)
#     print ("train_end: ", train_end)
#     print ("test_begin: ", test_begin)
#     print ("test_end: ", test_end)
#     print ("time_step: ", time_step)

    # 参数类别二：

    rnn_unit = rnn_unit # 1o as default  # 隐藏层单元数量 ##################调整这个
    input_size = 3  # 输入个数  
    output_size = 1  # 输出个数
    batch_size = batch_size # 80 as default  # 批量大小  #################调整这个
    train_times = train_times # 20 as default  # 训练次数
    lr = lr # 0.001 as default  # 学习率
    # 一般调整隐藏层数量、批量大小及学习率这几个超参数
    # 输入和输出则由特征量和标签确定
    # 本例中，以开盘价、最高价、最低价为特征量，
    # 以收盘价差即涨跌作为标签
    
  

    optimization_record = {'test_train_split': [],
                           'time_step': [],
                           'rnn_unit': [],
                           'batch_size': [],
                           'train_times':[],
                           'lr':[],
                           'total_rtrn_pct':[]}
    
    # Append参数：
    
    optimization_record['test_train_split'].append(test_train_split)
    optimization_record['time_step'].append(time_step)
    optimization_record['rnn_unit'].append(rnn_unit)
    optimization_record['batch_size'].append(batch_size)
    optimization_record['train_times'].append(train_times)
    optimization_record['lr'].append(lr)
    
    
    
    print ("="*60)
    def get_train_data(batch_size, time_step, train_begin, train_end):
        batch_index = []
        data_train = data[train_begin + 1:train_end]
        normalized_train_data = (
            data_train - np.mean(data_train, axis=0)) / np.std(
                data_train, axis=0)
        # logger.info('normalized_train_data.shape: ', normalized_train_data.shape)
        # logger.info(len(normalized_train_data))
        # logger.info(type(normalized_train_data))
        # logger.info(len([0, 2]))
        # logger.info(normalized_train_data[0:20])
        # logger.info(normalized_train_data[0:20, 1])

        train_x, train_y = [], []
        for i in range(len(normalized_train_data) - time_step):
            if i % batch_size == 0:
                batch_index.append(i)

            x = normalized_train_data[i:i + time_step, :3]
            y = normalized_train_data[i:i + time_step, 3, np.newaxis]
            train_x.append(x.tolist())
            train_y.append(y.tolist())

        batch_index.append(len(normalized_train_data) - time_step)
        return batch_index, train_x, train_y


    def get_test_data(time_step, test_begin, test_end):
        data_test = data[test_begin:test_end]
        test_y = data_test[:, 3]
        mean = np.mean(data_test, axis=0)
        std = np.std(data_test, axis=0)
        normalized_test_data = (data_test - mean) / std
        size = (len(normalized_test_data) + time_step - 1) // time_step
        test_x, test_y = [], []
        for i in range(size - 1):
            x = normalized_test_data[i * time_step:(i + 1) * time_step, :3]
            y = normalized_test_data[i * time_step:(i + 1) * time_step, 3]
            test_x.append(x.tolist())
            test_y.extend(y)
            # print('type(y): ', type(y))

        test_x.append((normalized_test_data[(i + 1) * time_step:, :3]).tolist())
        test_y.extend((normalized_test_data[(i + 1) * time_step:, 3]).tolist())
        return mean, std, test_x, test_y


    weights = {
        'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
        'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
    }
    biases = {
        'in': tf.Variable(tf.constant(0.1, shape=[
            rnn_unit,
        ])),
        'out': tf.Variable(tf.constant(0.1, shape=[
            1,
        ]))
    }


    # 定义神经网络变量
    def lstm(X):
        batch_size = tf.shape(X)[0]
        time_step = tf.shape(X)[1]
        w_in = weights['in']
        b_in = biases['in']
        input_ = tf.reshape(X, [-1, input_size])
        # print('input_.shape: ', input_.shape)
        input_rnn = tf.matmul(input_, w_in) + b_in
        # print('input_rnn.shape: ', input_rnn.shape)
        # tensor转换成3维，作为cell的输入
        input_rnn = tf.reshape(input_rnn, [-1, time_step, rnn_unit])
        # print('input_rnn.shape: ', input_rnn.shape)
        cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
        init_state = cell.zero_state(batch_size, dtype=tf.float32)
        output_rnn, final_state = tf.nn.dynamic_rnn(
            cell, input_rnn, initial_state=init_state, dtype=tf.float32)
        output = tf.reshape(output_rnn, [-1, rnn_unit])
        w_out = weights['out']
        b_out = biases['out']
        pred = tf.matmul(output, w_out) + b_out
        return pred, final_state


    # 训练模型
    def train_lstm(batch_size, time_step, train_begin, train_end):
        X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
        Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
        batch_index, train_x, train_y = get_train_data(batch_size, time_step,
                                                       train_begin, train_end)
        with tf.variable_scope('future_lstm'):
            pred, _ = lstm(X)
            loss = tf.reduce_mean(
                tf.square(tf.reshape(pred, [-1]) - tf.reshape(Y, [-1])))
            train_operation = tf.train.AdamOptimizer(lr).minimize(loss)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)

            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                for i in range(train_times):
                    for step in range(len(batch_index) - 1):
                        _, loss_ = sess.run(
                            [train_operation, loss],
                            feed_dict={
                                X:
                                train_x[batch_index[step]:batch_index[step + 1]],
                                Y: train_y[batch_index[step]:batch_index[step + 1]]
                            })
                    # print('Number of iterations: {} , loss: {}'.format(i, loss_))
#                     logger.info('Number of iterations: {} , loss: {}'.format(
#                         i, loss_))
    #             print('model saved: ', saver.save(sess, 'model_save1/model.ckpt'))
                print('The train has finished')


    train_lstm(batch_size, time_step, train_begin, train_end)


    def prediction_and_evaluate(time_step):
        X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    #     print ("X: ", X)  #brad's peek
        mean, std, test_x, test_y = get_test_data(time_step, test_begin, test_end)
        with tf.variable_scope('future_lstm', reuse=True):
            pred, _ = lstm(X)
            saver = tf.train.Saver(tf.global_variables())
            with tf.Session() as sess:
                model_file = tf.train.latest_checkpoint('model_save1')
                saver.restore(sess, model_file)
                test_predict = []
                # brad's peek:
    #             print("test_x")
    #             print (test_x)
                for step in range(len(test_x)):
                    # 一次出time_step个结果，results.shape: (20, 1)
                    results = sess.run(pred, feed_dict={X: [test_x[step]]})
                    # predict = tf.reshape(results, [-1])
                    predict = results.reshape(-1)
                    test_predict.extend(predict)
                # brad's peek:
    #             print (test_predict)
                # 还原真实值
                # brad's peek:
    #             print ("test_y",test_y)
                test_y = np.array(test_y) * std[3] + mean[3]
                test_predict = np.array(test_predict) * std[3] + mean[3]
                true_y = test_y


                # print('test_predict.shape: ', test_predict.shape)
                # print('true_y.shape: ', true_y.shape)
                out = np.c_[test_predict, true_y]
                out_csv = pd.DataFrame(
                    data=out, index=None, columns=['prediction', 'true'])
                right, wrong = 0, 0
                calc_data = map(lambda x, y: tuple((x, y)), test_predict,
                                test_y[:len(test_predict)])
                for data in calc_data:
                    if data[0] * data[1] > 0 or data[0] == data[1]:
                        right += 1
                    else:
                        wrong += 1
                accuracy = right / (right + wrong)
                print("利用过去一天: "+str(df[:test_end].iloc[-1, :].values[0]) + ", 来预测下一天：")
                print (out_csv.iloc[-1, 0])
                logger.info('The accuracy of this prediction: ')
                logger.info(accuracy)
        signal_file = out_csv.copy()
        signal_file.rename(columns={"true": "label"}, inplace = True)
        df_to_merge = df_asst_og[test_begin:].copy()
        signal_file['time'] = df_to_merge['time'].values
        merged = signal_file.merge(df_to_merge, on = 'time')
    #     del merged['label_x']
    #     del merged['label_y']
        merged = merged[['time', 'open', 'high', 'low', 'close', 'prediction', 'label_x']]

        def create_profits(x, y):
            if x * y > 0:
                return abs(y)
            else:
                return -abs(y)
        merged['profits'] = np.vectorize(create_profits)(merged['prediction'], merged['label_x'])
        merged['total_cum_profits'] = merged['profits'].cumsum()
        merged['total_cum_profits'].plot(figsize = (18, 6))
        return_pct = merged['total_cum_profits'].values[-1]/merged['close'].values[0]
        return return_pct

    target_to_optimize = prediction_and_evaluate(time_step)
    optimization_record['total_rtrn_pct'].append(target_to_optimize)
    print ("total_return_percent: ", target_to_optimize)
    return optimization_record

In [5]:
# This function will be mainly used on local environment for optimization's purpose, not for exporting data to TB.

def merge_and_optimize(asst, df, test_begin):
    # 分析强度信号：
    signal_loc = location
    signal_file = pd.read_csv(signal_loc + "pred_and_true_20181119_{}.csv".format(asst), engine='python')
    del signal_file['Unnamed: 0']
    signal_file.rename(columns={"true": "label"}, inplace = True)
    df_to_merge = df[test_begin:].copy()
    signal_file['time'] = df_to_merge['time'].values
    merged = signal_file.merge(df_to_merge, on = 'time')
#     del merged['label_x']
#     del merged['label_y']
    merged = merged[['time', 'open', 'high', 'low', 'close', 'prediction', 'label_x']]
    
    def create_profits(x, y):
        if x * y > 0:
            return abs(y)
        else:
            return -abs(y)
    merged['profits'] = np.vectorize(create_profits)(merged['prediction'], merged['label_x'])
    merged['total_cum_profits'] = merged['profits'].cumsum()
#     merged['total_cum_profits'].plot(figsize = (18, 6))
    return_pct = merged['total_cum_profits'].values[-1]/merged['close'].values[0]
    return return_pct

In [6]:
paras_to_optimize = {'test_train_split': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
                     'time_step': [20, 40],
                     'rnn_unit': [10, 20],
                     'batch_size': [],
                     'train_times':[],
                     'lr':[],
                     'total_rtrn_pct':[]}

In [7]:
import time

start = time.time()


results_list = []
for tts in paras_to_optimize['test_train_split']:
    for ts in paras_to_optimize['time_step']:
        for ru in paras_to_optimize['rnn_unit']:
            try:
                r = lstm_all_in_one_optimization(df_trx, df_trx, 'trx', 
                                             test_train_split = tts, 
                                             time_step = ts, 
                                             rnn_unit = ru, 
                                             batch_size = 80, 
                                             train_times = 20, 
                                             lr = 0.001)
                results_list.append(r)

            except:
                print ("Skipping the error paras")
            
            
end = time.time()
print(end - start)

The train has finished
Skipping the error paras
The train has finished
Skipping the error paras
The train has finished
Skipping the error paras


C:\Users\Brad Sun\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Brad Sun\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\Brad Sun\Anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Brad Sun\Anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Brad Sun\Anaconda3\lib\site-packages\numpy\core\_methods.py:130: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


The train has finished
Skipping the error paras
The train has finished
ERROR:tensorflow:Couldn't match files for checkpoint model_save1\model.ckpt
Skipping the error paras
The train has finished
ERROR:tensorflow:Couldn't match files for checkpoint model_save1\model.ckpt
Skipping the error paras
The train has finished
Skipping the error paras
The train has finished
Skipping the error paras
The train has finished
ERROR:tensorflow:Couldn't match files for checkpoint model_save1\model.ckpt
Skipping the error paras
The train has finished
ERROR:tensorflow:Couldn't match files for checkpoint model_save1\model.ckpt
Skipping the error paras
The train has finished
Skipping the error paras
The train has finished
Skipping the error paras
The train has finished
ERROR:tensorflow:Couldn't match files for checkpoint model_save1\model.ckpt
Skipping the error paras
The train has finished
ERROR:tensorflow:Couldn't match files for checkpoint model_save1\model.ckpt
Skipping the error paras
The train has fi

In [8]:
results_list

[]

In [9]:
df_results = pd.DataFrame(results_list)
df_results.sort_values('total_rtrn_pct', ascending=False)

KeyError: 'total_rtrn_pct'

In [36]:
df_results.to_csv(location+"trx_paras_results.csv")